In [23]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
import keras 
from keras.models import model_from_json
from skimage.transform import resize

In [24]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)
    img = ds.pixel_array
    img_mean = np.mean(img)
    img_std = np.std(img)
    return img, img_mean, img_std
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    proc_img = img.copy()
    proc_img = (proc_img - img_mean)/img_std
    proc_img = resize(img, img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    json_file = open(model_path, "r")
    loaded_model_json = json_file.read()
    json_file.close()
    model = model_from_json(loaded_model_json)
    model.load_weights(weight_path)
    return model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    pred_Y = model.predict(img, batch_size = 1, verbose = True)
    prediction = ''
    if pred_Y > thresh:
        prediction = 'Presence of Pneumonia'
    else:
        prediction = 'Absence of Pneumonia'
    return prediction 

In [26]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = "my_model.json" #path to saved model
weight_path = "my_model.hdf5" #path to saved best weights

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16

my_model = load_model(model_path, weight_path) #loads model
thresh = 0.35451794 #loads the threshold they chose for model classification 

# use the .dcm files to test your prediction
for i in test_dicoms:
    img = np.array([])
    img, img_mean, img_std = check_dicom(i)
    
    if img is None:
        continue
        
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Load file test1.dcm ...
1/1 [==============================] - 1s 581ms/step
Presence of Pneumonia
Load file test2.dcm ...
1/1 [==============================] - 1s 518ms/step
Presence of Pneumonia
Load file test3.dcm ...
1/1 [==============================] - 1s 534ms/step
Presence of Pneumonia
Load file test4.dcm ...
1/1 [==============================] - 1s 524ms/step
Presence of Pneumonia
Load file test5.dcm ...
1/1 [==============================] - 1s 516ms/step
Presence of Pneumonia
Load file test6.dcm ...
1/1 [==============================] - 1s 518ms/step
Presence of Pneumonia
